In [23]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import os
from selenium import webdriver
# Importation du gestionnaire de services pour Chrome
from selenium.webdriver.chrome.service import Service as ChromeService 
# Importation de l'outil pour télécharger/gérer le driver automatiquement
from webdriver_manager.chrome import ChromeDriverManager 
from PIL import Image
import time
# Nécessite que Pillow soit installé (pip install Pillow)
from PIL import Image


file_path = "data/incendies/incendies_V2.csv"

# Charger le fichier téléchargé sur data.gouv.fr
df_incendie = pd.read_csv(file_path)

C:\Users\simon\AppData\Local\Temp\ipykernel_19436\1310040949.py:19: DtypeWarning: Columns (16,19,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_incendie = pd.read_csv(file_path)


In [ ]:
# Assurez-vous que df_merged est prêt
df_incendie['latitude'] = pd.to_numeric(df_incendie['latitude'], errors='coerce')
df_incendie['longitude'] = pd.to_numeric(df_incendie['longitude'], errors='coerce')
df_incendie = df_incendie.dropna(subset=['latitude', 'longitude'])

OUTPUT_DIR = 'image/cartes_annuelles_feux_points'
os.makedirs(OUTPUT_DIR, exist_ok=True)

LAT_CENTER, LON_CENTER, ZOOM = 46.603354, 1.888334, 5

unique_years = sorted(df_incendie['Année'].unique())
print(f"Début de la génération des cartes pour {len(unique_years)} années...")

# Fonction pour colorer selon le nombre de feux
def get_color(n):
    if n <= 1:
        return 'blue'
    elif n <= 3:
        return 'lime'
    elif n <= 5:
        return 'orange'
    else:
        return 'red'

for year in unique_years:
    print(f"  -> Traitement de l'année {year}...")
    
    df_year = df_incendie[df_incendie['Année'] == year].copy()
    df_agg = df_year.groupby(['latitude', 'longitude']).size().reset_index(name='nombre_de_feux')
    
    # Création de la carte
    m = folium.Map(location=[LAT_CENTER, LON_CENTER], zoom_start=ZOOM, tiles="CartoDB positron")
    
    # Ajouter des points
    for idx, row in df_agg.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=6,
            color=get_color(row['nombre_de_feux']),
            fill=True,
            fill_color=get_color(row['nombre_de_feux']),
            fill_opacity=0.7,
            popup=f"Nombre de feux: {row['nombre_de_feux']}"
        ).add_to(m)
    
    # Ajouter titre
    title_html = f'<h3 align="center" style="font-size:20px"><b>Feux en France - {year}</b></h3>'
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Ajouter légende
    legend_html = """
     <div style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 120px; 
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;">
     <b>Légende</b><br>
     <i class="fa fa-circle" style="color:blue"></i> 1 feu<br>
     <i class="fa fa-circle" style="color:lime"></i> 2-3 feux<br>
     <i class="fa fa-circle" style="color:orange"></i> 4-5 feux<br>
     <i class="fa fa-circle" style="color:red"></i> >5 feux
     </div>
     """
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Sauvegarde
    output_path = os.path.join(OUTPUT_DIR, f'feux_{year}.html')
    m.save(output_path)

print("\nCartes annuelles générées dans :", OUTPUT_DIR)


Début de la génération des cartes pour 52 années...
  -> Traitement de l'année 1973...
  -> Traitement de l'année 1974...
  -> Traitement de l'année 1975...
  -> Traitement de l'année 1976...
  -> Traitement de l'année 1977...
  -> Traitement de l'année 1978...
  -> Traitement de l'année 1979...
  -> Traitement de l'année 1980...
  -> Traitement de l'année 1981...
  -> Traitement de l'année 1982...
  -> Traitement de l'année 1983...
  -> Traitement de l'année 1984...
  -> Traitement de l'année 1985...
  -> Traitement de l'année 1986...
  -> Traitement de l'année 1987...
  -> Traitement de l'année 1988...
  -> Traitement de l'année 1989...
  -> Traitement de l'année 1990...
  -> Traitement de l'année 1991...
  -> Traitement de l'année 1992...
  -> Traitement de l'année 1993...
  -> Traitement de l'année 1994...
  -> Traitement de l'année 1995...
  -> Traitement de l'année 1996...
  -> Traitement de l'année 1997...
  -> Traitement de l'année 1998...
  -> Traitement de l'année 1999...
  -

In [ ]:
# --- CONFIGURATION (Mise à jour) ---
IMG_DIR = 'image/images_png_feux'
os.makedirs(IMG_DIR, exist_ok=True)
# ---------------------------------------------------------

# 1. Initialisation du Driver Chrome
# On utilise ChromeDriverManager pour télécharger et utiliser le bon driver automatiquement
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service) 

driver.set_window_size(1000, 700) 

# Le reste de votre boucle (génération des PNG) :
for year in unique_years:
    html_path = os.path.abspath(os.path.join(OUTPUT_DIR, f'feux_{year}.html'))
    png_path = os.path.join(IMG_DIR, f'feux_{year}.png')
    
    print(f"  -> Rendu de l'année {year}...")
    
    # 1. Charger la page (via le protocole file:///)
    driver.get(f'file:///{html_path}')
    time.sleep(0.01) # Attendre que la carte Folium soit entièrement chargée

    # 2. Prendre la capture d'écran
    driver.save_screenshot(png_path)

driver.quit()

# ... Ensuite, le code pour créer le GIF avec PIL ...

  -> Rendu de l'année 1973...
  -> Rendu de l'année 1974...
  -> Rendu de l'année 1975...
  -> Rendu de l'année 1976...
  -> Rendu de l'année 1977...
  -> Rendu de l'année 1978...
  -> Rendu de l'année 1979...
  -> Rendu de l'année 1980...
  -> Rendu de l'année 1981...
  -> Rendu de l'année 1982...
  -> Rendu de l'année 1983...
  -> Rendu de l'année 1984...
  -> Rendu de l'année 1985...
  -> Rendu de l'année 1986...
  -> Rendu de l'année 1987...
  -> Rendu de l'année 1988...
  -> Rendu de l'année 1989...
  -> Rendu de l'année 1990...
  -> Rendu de l'année 1991...
  -> Rendu de l'année 1992...
  -> Rendu de l'année 1993...
  -> Rendu de l'année 1994...
  -> Rendu de l'année 1995...
  -> Rendu de l'année 1996...
  -> Rendu de l'année 1997...
  -> Rendu de l'année 1998...
  -> Rendu de l'année 1999...
  -> Rendu de l'année 2000...
  -> Rendu de l'année 2001...
  -> Rendu de l'année 2002...
  -> Rendu de l'année 2003...
  -> Rendu de l'année 2004...
  -> Rendu de l'année 2005...
  -> Rendu

In [ ]:
image_files = sorted([os.path.join(IMG_DIR, f) for f in os.listdir(IMG_DIR) if f.endswith('.png')])
images = [Image.open(f) for f in image_files]

# Création du GIF
images[0].save(
    'image/animation_feux_france.gif',
    save_all=True,
    append_images=images[1:],
    duration=500, # Durée de chaque image en millisecondes (1000 ms = 1 seconde)
    loop=0         # Boucle infinie (0)
)

print("\nGIF animé créé avec succès : animation_feux_france.gif")


GIF animé créé avec succès : animation_feux_france.gif
